In [1]:
import pandas as pd
import os
os.chdir("..")
#sys.path.append("/path/to/python/site-packages")
from utils.entity_extractor import EntityExtractor  
from utils.topic_modeling import TopicModeling

c:\Users\sofik\Documents\Uni Konstanz\SEDS\Thesis\master-thesis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test= pd.read_csv("data/processed/babe.csv").sample(300, random_state=42)
test.to_csv("data/processed/test.csv")

In [5]:
datasets_file_paths = {
    # "annomatic": "data/processed/annomatic.csv",
    # "babe": "data/processed/babe.csv",
    # "basil": "data/processed/basil.csv"
    "test": "data/processed/test.csv"
}
output_dir = "data/enriched/"
os.makedirs(output_dir, exist_ok=True)

# Initialize tools
entity_extractor = EntityExtractor()
topic_modeling = TopicModeling(output_dir)

def enrich_dataset(name, path):
    """
    Process a single dataset with NER and Topic Modeling, merging results on `text`.
    """
    # Load dataset
    print(f"Processing dataset: {name}")
    df = pd.read_csv(path)
    
    # Extract Entities
    print(f"Extracting entities for {name}...")
    processed_data = entity_extractor.process_data(df)

    # Cluster the entities
    df_entities = entity_extractor.cluster_entities(processed_data)
    
    # Topic Modeling
    print(f"Performing topic modeling for {name}...")
    
    # Process dataset with TopicModeling
    df_topics = topic_modeling.run(df, name)

    # Merge Results on `text`
    print(f"Merging results for {name}...")
    df_enriched = pd.merge(df_entities, df_topics, on="text", how="inner")
    
    # Save Enriched Dataset
    enriched_path = os.path.join(output_dir, f"{name}_full.parquet")
    df_enriched.to_parquet(enriched_path, index=False)
    print(f"Enriched dataset saved to: {enriched_path}")
    return df_enriched


Fetching 8 files: 100%|██████████| 8/8 [00:00<?, ?it/s]


2025-01-14 18:05:48,240 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Initializing TopicModeling class...
Initializing model...


In [6]:
for name, path in datasets_file_paths.items():
    enrich_dataset(name, path)    

Processing dataset: test
Extracting entities for test...


Processing Flair Entities: 100%|██████████| 300/300 [02:12<00:00,  2.26docs/s]


Performing topic modeling for test...


Fitting BERTopic model: 100%|██████████| 300/300 [00:00<00:00, 300882.64it/s]
2025-01-14 18:14:15,015 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 10/10 [00:03<00:00,  2.62it/s]
2025-01-14 18:14:18,883 - BERTopic - Embedding - Completed ✓
2025-01-14 18:14:18,884 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-14 18:14:31,866 - BERTopic - Dimensionality - Completed ✓
2025-01-14 18:14:31,873 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-14 18:14:31,913 - BERTopic - Cluster - Completed ✓
2025-01-14 18:14:31,927 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-14 18:14:32,012 - BERTopic - Representation - Completed ✓


Error in topic modeling for test: zero-size array to reduction operation maximum which has no identity
Merging results for test...
Enriched dataset saved to: data/enriched/test_full.parquet
